In [ ]:
# 🔧 Instalación de librerías necesarias (descomenta si es necesario en Colab)
#!pip install nltk scikit-learn

# 📚 Importar librerías
import nltk
import random
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 📥 Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

# 🧠 Base de conocimiento
base_conocimiento = {
    "liligo": {
        # General
        "¿Qué es LiliGo?": "Es un servicio de entregas rápidas, seguras y personalizadas que conecta pequeños negocios con sus clientes mediante una plataforma digital sencilla (web y WhatsApp).",
        "¿Cómo funciona LiliGo?": "El cliente solicita su domicilio desde la web o WhatsApp, puede programar la entrega, hacer seguimiento en tiempo real y recibir atención personalizada.",
        "¿Qué productos se pueden enviar?": "Alimentos, medicinas, documentos y encargos personales, siempre con transporte local y confiable.",

        # Formas de pago
        "¿Qué formas de pago acepta LiliGo?": "Aceptamos pago en efectivo al momento de la entrega 💵, transferencia bancaria 📲 y pago digital a través de aplicaciones autorizadas.",
        "¿Se puede pagar contraentrega?": "Sí, puedes pagar tu domicilio en efectivo cuando recibas el pedido.",
        "¿Los negocios pueden recibir pagos digitales?": "Sí, los negocios aliados pueden ofrecer pagos vía transferencia o billeteras digitales según su disponibilidad.",


        # Ubicación con sinónimos
        "¿Dónde está ubicada la empresa? / ¿Dónde están ubicados? / ¿Cuál es la sede de LiliGo?":
            "LiliGo nace en Cartago, Valle del Cauca 🇨🇴, y actualmente también funciona en La Virginia, Pereira, Dosquebradas y Armenia.",

        "¿En qué zonas funciona LiliGo? / ¿Qué ciudades cubre LiliGo? / ¿Dónde presta servicio LiliGo? / ¿Dónde trabajan?":
            "Actualmente operamos en Cartago, La Virginia, Pereira, Dosquebradas y Armenia 🏙️. Muy pronto estaremos disponibles en más ciudades del Valle y Eje Cafetero.",

        "¿Puedo usar LiliGo en cualquier lugar? / ¿Funciona en mi ciudad? / ¿Dónde puedo pedir un domicilio con LiliGo? / ¿LiliGo funciona en Armenia?":
            "Sí ✅, siempre que estés en una de nuestras zonas de cobertura activa: Cartago, La Virginia, Pereira, Dosquebradas o Armenia 🚴‍♂️📦."
    }

        # Horarios
        "¿En qué horarios funciona LiliGo?": "Nuestro servicio está disponible de lunes a sábado ⏰ en horario laboral. Las solicitudes pueden hacerse 24/7 desde la plataforma.",
        "¿Puedo pedir un domicilio en la noche?": "Sí, puedes programarlo, pero la entrega se realizará en los horarios activos de mensajería.",
        "¿El servicio funciona los domingos?": "En la actualidad solo ofrecemos entregas de lunes a sábado, pero estamos trabajando en expandir la cobertura.",

        # Envíos y logística
        "¿Quién realiza las entregas?": "Nuestros domiciliarios 🚴‍♂️ de confianza realizan cada envío con puntualidad y seguridad.",
        "¿Cómo se asegura la entrega?": "Todas las entregas cuentan con seguimiento en tiempo real 📍 y confirmación de recibido.",
        "¿Qué pasa si el mensajero no encuentra al cliente?": "Se intentará la entrega nuevamente o se coordinará con el cliente para una nueva entrega sin riesgos de pérdida.",

        # Costos
        "¿Cuánto cuesta un domicilio?": "El costo depende de la distancia y el tipo de pedido 📦. Siempre se informa antes de confirmar el servicio.",
        "¿Quién paga el envío?": "El cliente paga el domicilio. En algunos casos, los negocios pueden asumirlo como beneficio para sus clientes.",
        "¿LiliGo tiene tarifas adicionales?": "No, los precios son claros y sin costos ocultos. Buscamos mantener un servicio accesible y justo.",

        # Seguridad
        "¿Es seguro usar LiliGo?": "Sí 🔒, contamos con mensajeros verificados, seguimiento en tiempo real y comunicación directa vía plataforma.",
        "¿Cómo garantizan la confianza?": "Cada entrega incluye un sistema de verificación y confirmación, lo que asegura transparencia en todo el proceso.",

        # Atención al cliente
        "¿Cómo contacto al servicio al cliente?": "Puedes escribirnos desde la misma plataforma, WhatsApp 📲 o correo electrónico oficial.",
        "¿Qué hago si tengo un problema con mi pedido?": "Debes reportarlo en la plataforma y nuestro equipo dará una solución inmediata.",
        "¿Puedo calificar el servicio?": "Sí ⭐, puedes dejar una valoración para ayudarnos a mejorar y reconocer a los mejores domiciliarios."
    },

    # 🤝 Respuestas rápidas
    "respuestas_saludo": [
        "¡Hola! 👋 Soy LiliGo, tu asistente de entregas rápidas y seguras. 🚴‍♂️📦",
        "¡Bienvenido a LiliGo! 🌟 ¿Listo para enviar o recibir tu pedido hoy?",
        "¡Hola! 😊 Con LiliGo puedes conectar con tus negocios favoritos y recibir tus productos sin complicaciones.",
        "¡Hey! 🚀 Soy tu aliado en entregas rápidas, seguras y personalizadas. ¿Qué necesitas hoy?",
        "¡Bienvenido a LiliGo! 💖 Tu pedido a un clic de distancia"
    ],

    "respuestas_despedida": [
        "¡Gracias por confiar en LiliGo! 🚴‍♂️📦 Hasta pronto. 👋",
        "¡Tu entrega está en buenas manos! 🙌 Nos vemos pronto.",
        "¡Adiós! 🌟 LiliGo siempre cerca de ti.",
        "¡Hasta la próxima! 😊 Recuerda que con LiliGo tus pedidos llegan rápido y seguros.",
        "¡Nos vemos! 💖 Tu pedido a un clic de distancia"
    ]
}

# 🧹 Funciones de procesamiento de texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^\w\s¿?áéíóúñ]', ' ', texto)
    return ' '.join(texto.split())

def tokenizar_texto(texto):
    try:
        return nltk.word_tokenize(texto, language='spanish')
    except:
        return texto.split()

# 🎯 Funciones del chatbot
def detectar_saludo(pregunta):
    saludos = ['hola', 'buenos días', 'buenas tardes', 'buenas noches', 'saludos', 'qué tal', 'hey']
    return any(saludo in limpiar_texto(pregunta) for saludo in saludos)

def detectar_despedida(pregunta):
    despedidas = ['adiós', 'hasta luego', 'nos vemos', 'chau', 'bye', 'salir', 'terminar']
    return any(despedida in limpiar_texto(pregunta) for despedida in despedidas)

def calcular_similitud(pregunta_usuario, pregunta_base):
    try:
        vectorizador = CountVectorizer().fit([pregunta_usuario, pregunta_base])
        vectores = vectorizador.transform([pregunta_usuario, pregunta_base])
        return cosine_similarity(vectores[0], vectores[1])[0][0]
    except:
        palabras_usuario = set(tokenizar_texto(pregunta_usuario))
        palabras_base = set(tokenizar_texto(pregunta_base))
        return len(palabras_usuario & palabras_base) / max(len(palabras_usuario), len(palabras_base))

def encontrar_mejor_respuesta(pregunta_usuario):
    if detectar_saludo(pregunta_usuario):
        return random.choice(base_conocimiento["respuestas_saludo"])
    if detectar_despedida(pregunta_usuario):
        return random.choice(base_conocimiento["respuestas_despedida"])

    mejor_respuesta = ""
    mejor_puntuacion = 0
    materia_encontrada = ""

    for materia, preguntas in base_conocimiento.items():
        if not isinstance(preguntas, dict):
            continue

        for pregunta_base, respuesta in preguntas.items():
            puntuacion = calcular_similitud(limpiar_texto(pregunta_usuario), limpiar_texto(pregunta_base))
            if puntuacion > mejor_puntuacion:
                mejor_puntuacion = puntuacion
                mejor_respuesta = respuesta
                materia_encontrada = materia

    if mejor_puntuacion > 0.2:
        return f"📚 [{materia_encontrada.title()}] {mejor_respuesta}"
    else:
        return random.choice([
            "🤔 No estoy seguro, ¿puedes reformular la pregunta?",
            "🔍 ¿Cuál es tu consulta?",
            "❓ Hmm, no tengo esa información.",
            "😅 Lo siento, aún no sé cómo responder eso.",
            "📌 Intenta preguntarme de otra manera, por favor.",
            "🙋‍♂️ No entendí bien tu pregunta, ¿podrías repetirla?",
            "⚡ Estoy aprendiendo cada día, pero todavía no tengo esa respuesta.",
            "🤖 No tengo esa información exacta, pero puedo ayudarte con algo relacionado."
        ])

# 💬 Interfaz principal
def iniciar_chatbot():
    print("🤖 ChatBot LiliGo v1.0")
    print("Escribe 'salir' para terminar la conversación.\n")

    while True:
        usuario_input = input("🙋 Tú: ").strip()
        if not usuario_input:
            print("🤖 Bot: Por favor, escribe algo.")
            continue
        if detectar_despedida(usuario_input):
            print(f"🤖 Bot: {random.choice(base_conocimiento['respuestas_despedida'])}")
            break
        respuesta = encontrar_mejor_respuesta(usuario_input)
        print(f"🤖 Bot: {respuesta}")

# 🚀 Iniciar chatbot
iniciar_chatbot()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🤖 ChatBot LiliGo v1.0
Escribe 'salir' para terminar la conversación.

🙋 Tú: hola buenos dias 
🤖 Bot: ¡Hola! 😊 Con LiliGo puedes conectar con tus negocios favoritos y recibir tus productos sin complicaciones.
🙋 Tú: Quien es liligo
🤖 Bot: 📚 [Liligo] Es un servicio de entregas rápidas, seguras y personalizadas que conecta pequeños negocios con sus clientes mediante una plataforma digital sencilla (web y WhatsApp).
🙋 Tú: como funciona 
🤖 Bot: 📚 [Liligo] El cliente solicita su domicilio desde la web o WhatsApp, puede programar la entrega, hacer seguimiento en tiempo real y recibir atención personalizada.
🙋 Tú: donde se ubican 
🤖 Bot: 📚 [Liligo] Sí, puedes pagar tu domicilio en efectivo cuando recibas el pedido.
🙋 Tú: ubicacion 
🤖 Bot: 📌 Intenta preguntarme de otra manera, por favor.
🙋 Tú: la ubicacion 
🤖 Bot: 📚 [Liligo] Cada entrega incluye un sistema de verificación y confirmación, lo que asegura transparencia en todo el proceso.
🙋 Tú: y en que ciudades 
🤖 Bot: 📚 [Liligo] Actualmente opera